###### 2020-11-23 월요일

# 02_Bidirect LSTM
 - 구글 collab으로 실행하였습니다

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
import numpy as np
import re
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

## 1. 데이터불러오기 & 합치기

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

## 2. 데이터토큰화 & 패딩

#### 텍스트 음절단위로 자르기

In [ ]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

#### 정수인덱싱

In [ ]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

### 패딩

In [ ]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

## 3. SMOTE를 이용한 라벨 불균형 해소

In [ ]:
y_train.value_counts()

 - 라벨 0은 446162, 1은 2897, 2는 4993으로 균형이 잡혀있지않아 recall(재현율)이 떨어질 가능성이 높다
 - 그래서 SMOTE를 이용하여 라벨이 1과 2인 경우를 복제하여 라벨학습의 균형을 맞추어 주는 것이다
 - 이것을 `오버샘플링`이라 한다.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=123)
train_over, y_train_over = smote.fit_sample(train, y_train)

In [ ]:
pd.Series(y_train_over).value_counts()

## 4. Bidirect LSTM 구현

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score

In [ ]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 128
comment_len = 400

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size + 1, embedding_dim, input_length=comment_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 성능의 변화가 없을때 멈추는 기능
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

# 지금까지 가장 좋은 성능이 나왔을때, 노드의 가중치를 저장하는 함수
mc = ModelCheckpoint('erase_label_1.h5', monitor= 'val_loss', mode='min', save_best_only=True)

In [ ]:
y_train_over = y_train_over.reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [ ]:
print(train_over.shape)
print(test.shape)
print(y_train_over.shape)
print(y_test.shape)



In [ ]:
print(type(train_over))
print(type(test))
print(type(y_train_over))
print(type(y_test))

In [ ]:
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 20, 
                    batch_size       = 128, 
                    validation_split = 0.3)

## 5. Test Set 예측
 - confusion matrix의 대각선 원소의 수가 많아지도록 모델을 짜주십쇼

In [ ]:
# 저장한 LSTM 모델을 불러오고 예측해보자

loaded_model = load_model('erase_label_1.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5
confusion_matrix(y_class, y_test)

array([[11057,   497],
       [  511,   722]])

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

print('recall : ', recall_score(y_class, y_test))
print('precision : ', precision_score(y_class, y_test))
print('f1_score : ', f1_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))

recall :  0.5855636658556367
precision :  0.5922887612797375
f1_score :  0.5889070146818923
roc_auc_score :  0.893694679220321


## 5. 모델 테스트

In [ ]:
def model_test(comment):
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [ ]:
comment = ''''''

result, prob = model_test(comment)